In [1]:
import tweepy
import pandas as pd
import numpy as np
import json
import time
import shutil

In [2]:
archive = pd.read_csv('twitter-archive-enhanced.csv')
archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [3]:
archive[archive['retweeted_status_id'].notnull()]

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
19,888202515573088257,NaN,NaN,2017-07-21 01:02:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Canela. She attempted s...,8.874740e+17,4.196984e+09,2017-07-19 00:47:34 +0000,https://twitter.com/dog_rates/status/887473957...,13,10,Canela,None,None,None,None
32,886054160059072513,NaN,NaN,2017-07-15 02:45:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,8.860537e+17,1.960740e+07,2017-07-15 02:44:07 +0000,https://twitter.com/dog_rates/status/886053434...,12,10,None,None,None,None,None
36,885311592912609280,NaN,NaN,2017-07-13 01:35:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Lilly. She just paralle...,8.305833e+17,4.196984e+09,2017-02-12 01:04:29 +0000,https://twitter.com/dog_rates/status/830583320...,13,10,Lilly,None,None,None,None
68,879130579576475649,NaN,NaN,2017-06-26 00:13:58 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Emmy. She was adopted t...,8.780576e+17,4.196984e+09,2017-06-23 01:10:23 +0000,https://twitter.com/dog_rates/status/878057613...,14,10,Emmy,None,None,None,None
73,878404777348136964,NaN,NaN,2017-06-24 00:09:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Shadow. In an attempt to r...,8.782815e+17,4.196984e+09,2017-06-23 16:00:04 +0000,"https://www.gofundme.com/3yd6y1c,https://twitt...",13,10,Shadow,None,None,None,None
74,878316110768087041,NaN,NaN,2017-06-23 18:17:33 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: Meet Terrance. He's being yelle...,6.690004e+17,4.196984e+09,2015-11-24 03:51:38 +0000,https://twitter.com/dog_rates/status/669000397...,11,10,Terrance,None,None,None,None
78,877611172832227328,NaN,NaN,2017-06-21 19:36:23 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @rachel2195: @dog_rates the boyfriend and h...,8.768508e+17,5.128045e+08,2017-06-19 17:14:49 +0000,https://twitter.com/rachel2195/status/87685077...,14,10,None,None,None,pupper,None
91,874434818259525634,NaN,NaN,2017-06-13 01:14:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Coco. At first I though...,8.663350e+17,4.196984e+09,2017-05-21 16:48:45 +0000,https://twitter.com/dog_rates/status/866334964...,12,10,Coco,None,None,None,None
95,873697596434513921,NaN,NaN,2017-06-11 00:25:14 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Walter. He won't start ...,8.688804e+17,4.196984e+09,2017-05-28 17:23:24 +0000,https://twitter.com/dog_rates/status/868880397...,14,10,Walter,None,None,None,None
97,873337748698140672,NaN,NaN,2017-06-10 00:35:19 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: This is Sierra. She's one preci...,8.732138e+17,4.196984e+09,2017-06-09 16:22:42 +0000,https://www.gofundme.com/help-my-baby-sierra-g...,12,10,Sierra,None,None,pupper,None


In [358]:
consumer_key = 'FZIv8QjifocEY8jpGeUoR5w91'
consumer_secret = 'aJiVzmuSmpsQ4ivcmISrWUQWM4lr4aQdAaBvcVWiFrjKqIu7Yc'
access_token = '3152304601-n2xOfrExr0psH5SvwRq7V8ywCF7FuO8OH4gxaBv'
access_secret = '8Sbc19WdM0J9iD0UnvIbsALLT5sPDFhFGIkuEVQ67sDoK'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [359]:
sample_id = archive['tweet_id']

In [289]:
#sample_id = archive['tweet_id'].sample(100)

In [271]:
#sample_id = [780092040432480260]

In [360]:
sample_id.shape

(2356,)

In [361]:
start_time = time.time()

errors = []
df_list = []
count = 0

for one_id in sample_id:
    count += 1
    try:
        tweet = api.get_status(one_id, tweet_mode='extended')
        tweet_json = tweet._json
        if count % 10 == 0:
            print('{}. ID {} and time elapsed {:0.3f} secs'.format(count, one_id, time.time()-start_time))
        
        with open('tweet_json.txt','a') as outfile:
            json.dump(tweet_json, outfile)
            outfile.write('\n')
            
    except Exception as e:
        errors.append([one_id,e])
        if len(errors) == 100:
            break
        continue
        

10. ID 890240255349198849 and time elapsed 4.964 secs
30. ID 886366144734445568 and time elapsed 14.887 secs
40. ID 884876753390489601 and time elapsed 19.850 secs
50. ID 882762694511734784 and time elapsed 24.814 secs
60. ID 880872448815771648 and time elapsed 29.842 secs
70. ID 879050749262655488 and time elapsed 34.802 secs
80. ID 877556246731214848 and time elapsed 39.967 secs
90. ID 875021211251597312 and time elapsed 45.021 secs
100. ID 872967104147763200 and time elapsed 50.951 secs
110. ID 871166179821445120 and time elapsed 55.886 secs
120. ID 869772420881756160 and time elapsed 60.895 secs
130. ID 867421006826221569 and time elapsed 65.844 secs
140. ID 865359393868664832 and time elapsed 70.895 secs
150. ID 863079547188785154 and time elapsed 75.798 secs
160. ID 860981674716409858 and time elapsed 80.836 secs
170. ID 859196978902773760 and time elapsed 85.780 secs
180. ID 857214891891077121 and time elapsed 90.772 secs
190. ID 855860136149123072 and time elapsed 95.752 secs
2

Rate limit reached. Sleeping for: 442


1460. ID 695064344191721472 and time elapsed 1191.726 secs
1470. ID 694183373896572928 and time elapsed 1196.715 secs
1480. ID 693582294167244802 and time elapsed 1201.642 secs
1490. ID 692905862751522816 and time elapsed 1206.586 secs
1500. ID 692187005137076224 and time elapsed 1211.443 secs
1510. ID 691459709405118465 and time elapsed 1216.429 secs
1520. ID 690735892932222976 and time elapsed 1221.485 secs
1530. ID 690248561355657216 and time elapsed 1226.605 secs
1540. ID 689661964914655233 and time elapsed 1231.503 secs
1550. ID 689255633275777024 and time elapsed 1236.629 secs
1560. ID 688547210804498433 and time elapsed 1241.544 secs
1570. ID 687807801670897665 and time elapsed 1246.553 secs
1580. ID 687312378585812992 and time elapsed 1251.417 secs
1590. ID 686683045143953408 and time elapsed 1259.291 secs
1600. ID 686034024800862208 and time elapsed 1264.399 secs
1610. ID 685547936038666240 and time elapsed 1269.332 secs
1620. ID 684959798585110529 and time elapsed 1274.409 se

Rate limit reached. Sleeping for: 445


In [362]:
print(errors)

[[888202515573088257, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [873697596434513921, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [872668790621863937, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [872261713294495745, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [869988702071779329, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [866816280283807744, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [861769973181624320, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [856602993587888130, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [851953902622658560, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [845459076796616705, TweepError([{'code': 144, 'message': 'No status found with that ID.'}],)], [844704788403113984, TweepError([{'code

In [363]:
len(errors)

21

### Loading the downloaded file with all tweets as JSON each in its own line

In [4]:
archive_data = []

with open('tweet_json.txt') as json_file:  
    for line in json_file:
        archive_data.append(json.loads(line))

In [5]:
len(archive_data)

2335

In [7]:
# To be run together with twitter API call to check if statuses for all twitter IDs where received
len(sample_id)-len(archive_data)-len(errors)

In [40]:
df_list = []

for item in archive_data:
    try:
        dog_pic = item['entities']['media'][0]['media_url']
    except:
        try:
            dog_pic = item['retweeted_status']['entities']['media'][0]['media_url']
        except:
            dog_pic = 'None'

    try:    
        text = item['full_text']
    except:    
        text = 'None'

    try:    
        tweet_url = item['entities']['media'][0]['expanded_url']
    except:
        try:
            tweet_url = return_json['retweeted_status']['entities']['media'][0]['expanded_url']
        except:
            tweet_url = 'None'

    try:    
        fav_count = item['favorite_count']
        assert fav_count > 0
    except AssertionError:
        fav_count = item['retweeted_status']['favorite_count']
    except Exception:    
        fav_count = 'None'

    try:    
        retwt_count = item['retweet_count']
    except:
        retwt_count = 'None'

    try:    
        retwt_status_id = item['retweeted_status']['id']
    except:
        retwt_status_id = 'None'

    

    df_list.append({'tweet_id': item['id'],
                    'text_check': text,
                    'dog_picture_url': dog_pic,
                    'tweet_web_url' : tweet_url,
                    'favourite_count': fav_count,
                    'retweet_count': retwt_count,
                    'retweet_status_id_api': retwt_status_id})

df = pd.DataFrame(df_list, columns = ['tweet_id', 'text_check', 'dog_picture_url', 'tweet_web_url', 
                                      'favourite_count', 'retweet_count', 'retweet_status_id_api'])

In [41]:
df.head()

,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api
0,892420643555336193,This is Phineas. He's a mystical boy. Only eve...,http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,https://twitter.com/dog_rates/status/892420643...,37434,8138,None
1,892177421306343426,This is Tilly. She's just checking pup on you....,http://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,https://twitter.com/dog_rates/status/892177421...,32164,6029,None
2,891815181378084864,This is Archie. He is a rare Norwegian Pouncin...,http://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,https://twitter.com/dog_rates/status/891815181...,24251,3991,None
3,891689557279858688,This is Darla. She commenced a snooze mid meal...,http://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,https://twitter.com/dog_rates/status/891689557...,40731,8304,None
4,891327558926688256,This is Franklin. He would like you to stop ca...,http://pbs.twimg.com/media/DF6hr6AVYAAZ8G8.jpg,https://twitter.com/dog_rates/status/891327558...,38968,8992,None


In [42]:
df[df.duplicated()]

,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api


In [336]:
#df = df[~df.duplicated()]

In [43]:
sum(df.duplicated())

0

In [44]:
df[df.dog_picture_url == 'None']

,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api
29,886267009285017600,@NonWhiteHat @MayhewMayhem omg hello tanner yo...,None,https://twitter.com/dog_rates/status/830583320...,116,4,None
31,886054160059072513,RT @Athletics: 12/10 #BATP https://t.co/WxwJmv...,None,https://twitter.com/dog_rates/status/830583320...,1501,105,886053734421102592
34,885518971528720385,I have a new hero and his name is Howard. 14/1...,None,https://twitter.com/dog_rates/status/830583320...,19777,3596,None
41,884247878851493888,OMG HE DIDN'T MEAN TO HE WAS JUST TRYING A LIT...,None,https://twitter.com/dog_rates/status/830583320...,70538,19726,None
54,881633300179243008,@roushfenway These are good dogs but 17/10 is ...,None,https://twitter.com/dog_rates/status/830583320...,123,7,None
63,879674319642796034,@RealKentMurphy 14/10 confirmed,None,https://twitter.com/dog_rates/status/830583320...,302,10,None
71,878604707211726852,Martha is stunning how h*ckin dare you. 13/10 ...,None,https://twitter.com/dog_rates/status/830583320...,29243,6937,None
82,876537666061221889,I can say with the pupmost confidence that the...,None,https://twitter.com/dog_rates/status/830583320...,22835,4481,None
87,875097192612077568,You'll get your package when that precious man...,None,https://twitter.com/dog_rates/status/830583320...,26593,5853,None
106,871102520638267392,Never doubt a doggo 14/10 https://t.co/AbBLh2FZCH,None,https://twitter.com/dog_rates/status/830583320...,20392,5380,None


In [45]:
df[df.retweet_status_id_api == 'None'].count()

tweet_id                 2170
text_check               2170
dog_picture_url          2170
tweet_web_url            2170
favourite_count          2170
retweet_count            2170
retweet_status_id_api    2170
dtype: int64

In [46]:
df[df.tweet_web_url == 'None']

,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api


In [47]:
df.favourite_count.sort_values()

1275        51
329         68
2318        75
2277        82
2298        99
2316        99
2330       103
2306       103
2325       108
2321       110
1060       112
29         116
109        118
2304       118
2299       119
2332       120
54         123
2333       124
2329       129
2323       140
2314       141
279        145
2168       146
2319       148
2320       158
2302       159
2300       160
2307       161
2278       163
2315       167
         ...  
90       51583
851      51617
158      51617
607      52483
444      54263
290      54263
380      54364
351      55110
7        63112
118      64097
22       66651
343      69411
35       69411
41       70538
148      73557
37       74829
1809     76676
76       77148
521      78945
111      80792
1745     81065
162      89278
431      90844
434      90844
65      102479
1059    119452
129     120097
518     124911
398     137875
1019    161664
Name: favourite_count, Length: 2335, dtype: int64

In [48]:
df[df.favourite_count == 'None']

C:\Users\vilij\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api


In [49]:
df[df.retweet_count == 'None']

C:\Users\vilij\Anaconda3\lib\site-packages\pandas\core\ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api


In [50]:
df[df.text_check == 'None']

,tweet_id,text_check,dog_picture_url,tweet_web_url,favourite_count,retweet_count,retweet_status_id_api


In [51]:
for item in archive_data:
    if item['id'] == 885311592912609280:
        return_json = item
return_json

{'contributors': None,
 'coordinates': None,
 'created_at': 'Thu Jul 13 01:35:06 +0000 2017',
 'display_text_range': [0, 133],
 'entities': {'hashtags': [],
  'media': [{'display_url': 'pic.twitter.com/SATN4If5H5',
    'expanded_url': 'https://twitter.com/dog_rates/status/830583320585068544/photo/1',
    'id': 830583314243268608,
    'id_str': '830583314243268608',
    'indices': [110, 133],
    'media_url': 'http://pbs.twimg.com/media/C4bTH6nWMAAX_bJ.jpg',
    'media_url_https': 'https://pbs.twimg.com/media/C4bTH6nWMAAX_bJ.jpg',
    'sizes': {'large': {'h': 2048, 'resize': 'fit', 'w': 1536},
     'medium': {'h': 1200, 'resize': 'fit', 'w': 900},
     'small': {'h': 680, 'resize': 'fit', 'w': 510},
     'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
    'source_status_id': 830583320585068544,
    'source_status_id_str': '830583320585068544',
    'source_user_id': 4196983835,
    'source_user_id_str': '4196983835',
    'type': 'photo',
    'url': 'https://t.co/SATN4If5H5'}],
  'symbo

In [53]:
for item in archive_data:
    try:
        if item['retweeted_status']['id']:
            return_json = item
        break
    except:
        continue
return_json

{'contributors': None,
 'coordinates': None,
 'created_at': 'Sat Jul 15 02:45:48 +0000 2017',
 'display_text_range': [0, 50],
 'entities': {'hashtags': [{'indices': [21, 26], 'text': 'BATP'}],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/dog_rates/stat…',
    'expanded_url': 'https://twitter.com/dog_rates/status/886053434075471873',
    'indices': [27, 50],
    'url': 'https://t.co/WxwJmvjfxo'}],
  'user_mentions': [{'id': 19607400,
    'id_str': '19607400',
    'indices': [3, 13],
    'name': "Oakland A's",
    'screen_name': 'Athletics'}]},
 'favorite_count': 0,
 'favorited': False,
 'full_text': 'RT @Athletics: 12/10 #BATP https://t.co/WxwJmvjfxo',
 'geo': None,
 'id': 886054160059072513,
 'id_str': '886054160059072513',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': True,
 'lang': 'und',
 'place': None,
 'possibly_sensitive': False,


In [56]:
return_json['retweeted_status']['id']

886053734421102592

In [389]:
#shutil.copy('tweet_json.txt', 'tweet_json_mastercopy.txt')

'tweet_json_mastercopy.txt'

In [58]:
df.to_csv('twitter_status_dogs.csv',index=False)

In [31]:
#download image classification results table
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
df_class = pd.read_csv(url, sep='\t')
df_class.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [32]:
df_class.shape

(2075, 12)

In [33]:
df_class.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [34]:
df_class.to_csv('twitter_image_class.csv',index=False)